 使用pytorch搭建神经网络模型，实现对KMNIST数据集的训练

In [85]:
#1.导入必要的包
import torch
import torch.nn as nn                           #模型工具包
import torch.optim as optim                     #优化器
from torchvision.transforms.v2 import ToTensor  #转换图像为张量
from torchvision.datasets import KMNIST         #数据集
from torch.utils.data import DataLoader         #数据加载器 ，优化大量数据

In [142]:
#2.定义超参数
LR = 1e-2
epochs = 50
BATCH_SIZE = 128 #数据批次拆分

In [143]:
###2.数据集加载
train_data1 = KMNIST(root = './data',train = True ,download = True,transform = ToTensor())
test_data1 = KMNIST(root = './data',train = False ,download = True,transform = ToTensor())

In [144]:
#3.数据拆分
train_dl1 = DataLoader(train_data1,batch_size= BATCH_SIZE,shuffle = True) #shuffle 随机打乱拆分

In [145]:
#3.2张量参数确认
train_data1[1][0].reshape(-1).shape #将图像转为1维向量

torch.Size([784])

In [146]:
#3.3类别确认
labels = set( [clz for img,clz in train_data1 ])  #类别索引
labels

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [147]:
##4.模型的定义 增加了一个隐藏层
model = nn.Sequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,10)
)

In [148]:
##5.损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [149]:
##6.模型的训练
for epoch in range(epochs):
    #提取训练的数据  数据导入到模型中
    for data,target in train_dl1:
        out_put = model(data.reshape(-1,784))           #前向运算/前向传播
        loss = loss_fn(out_put,target)  #计算损失
        #反向传播 
        optimizer.zero_grad()           #所有参数梯度清零
        loss.backward()                 #计算梯度(参数，grad)
        optimizer.step()                #更新参数
        print(f'Epoch:{epoch},Loss:{loss.item()}')

Epoch:0,Loss:2.3107266426086426
Epoch:0,Loss:2.3084304332733154
Epoch:0,Loss:2.3041276931762695
Epoch:0,Loss:2.3002259731292725
Epoch:0,Loss:2.3079540729522705
Epoch:0,Loss:2.2978577613830566
Epoch:0,Loss:2.315636157989502
Epoch:0,Loss:2.300143003463745
Epoch:0,Loss:2.3121700286865234
Epoch:0,Loss:2.3093671798706055
Epoch:0,Loss:2.3006997108459473
Epoch:0,Loss:2.3031346797943115
Epoch:0,Loss:2.300112247467041
Epoch:0,Loss:2.310730218887329
Epoch:0,Loss:2.3140833377838135
Epoch:0,Loss:2.326853036880493
Epoch:0,Loss:2.299821138381958
Epoch:0,Loss:2.2912018299102783
Epoch:0,Loss:2.2947909832000732
Epoch:0,Loss:2.307361602783203
Epoch:0,Loss:2.2925002574920654
Epoch:0,Loss:2.3124735355377197
Epoch:0,Loss:2.2901453971862793
Epoch:0,Loss:2.305654287338257
Epoch:0,Loss:2.2958576679229736
Epoch:0,Loss:2.308016538619995
Epoch:0,Loss:2.3214173316955566
Epoch:0,Loss:2.3133111000061035
Epoch:0,Loss:2.3014721870422363
Epoch:0,Loss:2.3008158206939697
Epoch:0,Loss:2.301419734954834
Epoch:0,Loss:2.303

In [150]:
##6.0测试
test_dl = DataLoader(train_data1,batch_size= BATCH_SIZE)
correct = 0
total = 0
with torch.no_grad():    #不进行梯度计算
    for data,target in test_dl:
        output = model(data.reshape(-1,784)) 
        _,predicted = torch.max(output,1)
        total += target.size(0) #size(0) = shape[0]
        correct+= (predicted == target).sum().item()
print(f'{correct/total*100}%')      

99.35833333333333%


In [ ]:
#第一次训练.LR = 1e-2，epochs = 20，BATCH_SIZE = 128  准确率53.7%
#第二次训练.LR = 1e-2，epochs = 30，BATCH_SIZE = 64 隐藏层中间神经元数量256  准确率78.92%
#第三次训练.LR = 1e-2，epochs = 50，BATCH_SIZE = 64 隐藏层中间神经元数量256  准确率84.49%
#第四次训练.LR = 1e-2，epochs = 30，BATCH_SIZE = 64 隐藏层中间神经元数量256  nn.Sigmoid()替换为nn.ReLU() 准确率93.44%
#第五次训练 在四的基础上再新增一个隐藏层神经元为128  准确率95.02%
#第六次训练 在五的基础上批次增加到50  准确率98.19%
#第七次训练 在六的基础上再新增一个隐藏层神经元为32  准确率99.35%